# [EI-DA 전자산업 판매 데이터 분석]  Power Mart Electronic & Furniture

**<div style="text-align: right">Update date: April 15, 2022.</div>**
**<div style="text-align: right">Copyright(c) 2021-2022 Youngmin Park All rights reserved.</div>**

### DESCRIPTION  
Power Mart sample sales data  
### SUMMARY  
Power Mart is an electronic & furniture retailer wirth a strong international presence in Asia. In 2012, Power Mart entered in North American market by setting up shop in Canada. Power Mart has plans to expand into USA. However, the current economic climate does not look favorable. Nonetheless, Power Mart feels it is still in a good position to enter American market.


### Purpose of the data

This dataset is to visualize & design the dashboard to provide management with insights into how Power Mart's product mix contribute to the company's overall profitability. This will guide management decision making in their growth & expansion plans in Canada & USA.

The aim's to answer the following questions:

- a) Which are the most sellable products

- b) Which are the most profitable products

- c) Which products are underperforming

- d) Who are buying our products

- e) Which product segment should we focus on to spearhead growth

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# 한글 폰트 설정 (윈도우 Malgun Gothic,  리눅스&Mac AppltGothic)
import matplotlib.font_manager as fm
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
plt.rc('font', family='NanumBarunGothic') 

# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_file_path = '/content/drive/MyDrive/Colab Notebooks/DataVisualization_python/data/PowerMart.xlsx'

In [ ]:
orders_df = pd.read_excel(data_file_path, sheet_name='Orders')
returns_df = pd.read_excel(data_file_path, sheet_name='Returns')
users_df = pd.read_excel(data_file_path, sheet_name='Users')

##### orders_df 살펴보기

In [ ]:
orders_df.shape

In [ ]:
orders_df.head()

In [ ]:
orders_df.columns

##### returns_df 살펴보기

In [ ]:
returns_df.shape

In [ ]:
returns_df.head()

In [ ]:
returns_df.columns

returns_df에서 Order Id 컬럼이 orders_df와 연결됩니다

##### users_df 살펴보기

In [ ]:
users_df.shape

In [ ]:
users_df.head()

In [ ]:
users_df.columns

users_df에서 Region 컬럼이 orders_df와 연결됩니다

# 데이터 전처리  
- 이후 데이터 분석을 편하게 수행하기 위해, 세 dataframe을 하나로 병합합니다.
- returns_df는 'Order ID' 기준으로 orders_df와 병합합니다.
- users_df는 'Region' 기준으로 orders_df와 병합합니다.

## returns_df 병합

In [ ]:
_left = orders_df.set_index('Order ID')

In [ ]:
_right = returns_df.set_index('Order ID')

In [ ]:
powermart_df = pd.concat([_left, _right], axis=1).reset_index()

In [ ]:
powermart_df['Status'] = powermart_df['Status'].fillna('Completed')  # 반품되지 않은 경우에 대해 값 저장

In [ ]:
powermart_df

##users_df 병합

In [ ]:
users_df

하나의 Region에 여러명의 Manager가 있으므로, 제대로 concat되지 않습니다.  
따라서, users_df의 모양을 처리가 가능하도록 변경합니다.

In [ ]:
'Pat,' + users_df.groupby('Region').first()

In [ ]:
_right = 'Pat,' + users_df.groupby('Region').first()
_right = _right.reset_index()

In [ ]:
powermart_df = powermart_df.merge(_right, left_on='Region', right_on='Region', how='left')

In [ ]:
powermart_df = powermart_df.set_index('Row ID')

In [ ]:
powermart_df

병합 완료, powermart_df에 orders_df와 returns_df에 있던 컬럼들이 잘 붙은 것을 확인할 수 있습니다.

# EDA 탐색적 데이터 분석 수행하기

#### .info() 함수로 데이터 컬럼별 타입(자료형), 값이 있는 행(Non-Null)의 갯수 보기

In [ ]:
powermart_df.shape

In [ ]:
powermart_df.info()

#### 데이터를 직접 살펴보고 어떻게 분석해야 할지 생각해보기

In [ ]:
samples = powermart_df.sample(10)

In [ ]:
# 데이터 컬럼 전반부
samples.iloc[:,0:8]

In [ ]:
# 데이터 컬럼 중반부
samples.iloc[:,8:16]

In [ ]:
# 데이터 컬럼 후반부
samples.iloc[:,16:24]

#### 컬럼을 형태 타입별로 나누기

컬럼들을 반복문으로 분석하기 위해  
범주형 컬럼 (category_cols), 수치형 컬럼 (numerical_cols), 고유값 컬럼(value_cols)로 분류하여 리스트를 만들어봅니다.

In [ ]:
category_cols = ['Order Priority','Ship Mode','State','Region','Customer Segment','Product Category','Product Sub-Category',
                'Product Container','Status','Manager']

In [ ]:
numerical_cols = ['Order Quantity','Sales','Discount','Profit','Unit Price','Shipping Cost','Product Base Margin']

In [ ]:
value_cols = ['Customer Name','City','Zip Code','Product Name']

.describe() 함수로 수치형 컬럼에 대한 대략적인 통계 값들을 봅니다. (평균, 상위 25/50/75% 값, 최대/최소 값 등)

In [ ]:
powermart_df.loc[:,numerical_cols].describe()

- Profit의 경우, 중위값은 -1.5로 적자입니다. 그러나 평균이 181.18로 높습니다
- 이는 대부분의 작은 거래에서 적자를 보지만, 몇몇개의 거래들이 큰 수익을 가져다 주기 때문일 것이라고 추측할 수 있습니다

.value_counts()를 통해 각 카테고리 컬럼별로 어떻게 데이터가 있는지 살펴봅니다.

In [ ]:
for col in category_cols:
    print(col + " 카운트::")
    print(powermart_df.loc[:, col].value_counts(dropna=False))
    print()

.value_counts()를 통해 각 고유값 컬럼별로 어떻게 데이터가 있는지 살펴봅니다.

In [ ]:
for col in value_cols:
    print(col + " 카운트::")
    print(powermart_df.loc[:, col].value_counts(dropna=False))
    print()

#### 카테고리 컬럼별로 countplot 그려보기

In [ ]:
figure, ax_list_list = plt.subplots(nrows=5, ncols=2);
figure.set_size_inches(10,12)

ax_list = ax_list_list.reshape(10)  # 다차원 행렬의 차원을 원하는 모양으로 변경합니다.
print(ax_list_list.shape)
print(ax_list.shape)

for i in range(len(category_cols)):
    col = category_cols[i]
    sns.countplot(data=powermart_df, x=col, ax=ax_list[i])
    ax_list[i].set_title(col)

plt.tight_layout()

보기 힘든 컬럼들에 대해 제각기 그래프를 그려보기

In [ ]:
plt.figure(figsize = (12,5))
plt.xticks(rotation=90)
sns.countplot(data=powermart_df, x='Product Sub-Category');

- 가장 많은 주문건수를 나타내는 TOP 3 품목들은 [Paper, Binders and +, Telephones and Comm] 입니다
- 가장 적은 주문건수를 나타내는 TOP 3 품목들은 [Copiers and Fax, Scissors +, Rubber Bands] 입니다

In [ ]:
plt.figure(figsize = (12,5))
plt.xticks(rotation=90)
sns.countplot(data=powermart_df, x='State');

- 주문 건수가 가장 많은 TOP 3 State는 [California, Texas, Illinois] 입니다

In [ ]:
plt.figure(figsize = (12,5))
plt.xticks(rotation=90)
sns.countplot(data=powermart_df, x='Product Container');

- 대부분의 상품들은 작은 박스에 포장되어 판매된 것을 알 수 있습니다

#### 수치형 컬럼별로 distplot 그리기

In [ ]:
len(numerical_cols)

In [ ]:
figure, ax_list_list = plt.subplots(nrows=4, ncols=2);
figure.set_size_inches(10,16)

ax_list = ax_list_list.reshape(8)  # 다차원 행렬의 차원을 원하는 모양으로 변경합니다.
print(ax_list_list.shape)
print(ax_list.shape)

for i in range(len(numerical_cols)):
    col = numerical_cols[i]
    sns.histplot(data=powermart_df, x=col, ax=ax_list[i])
    ax_list[i].set_title(col)

plt.tight_layout()

In [ ]:
category_cols, numerical_cols, value_cols

#### 주문 날짜에 대한 히스토그램 그려보기

In [ ]:
powermart_df['Order Date'].hist(figsize=(12,6), bins=100);

어느정도 차이가 있긴 하나, 전반적으로 균등하게 분포하는 것을 알 수 있습니다

# 데이터로부터 유의미한 정보 발굴하기

### 제품군을 기준으로 분석

In [ ]:
powermart_df[['Order Quantity', 'Sales', 'Profit', 'Shipping Cost']].sum()

In [ ]:
# 1.234567e+10의 지수형태로 나타내서 보기 불편합니다. (과학적 표기법)
# 일반적인 출력 포맷으로 pandas 옵션을 변경해봅니다.
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
powermart_df[['Order Quantity', 'Sales', 'Profit', 'Shipping Cost']].sum()

전체 규모에 대해 가늠하기 위해 sum으로 계산해 보았습니다

In [ ]:
# 주문 건수가 가장 많은 제품군
powermart_df.groupby('Product Sub-Category')['Order ID'].count().sort_values().plot(kind='bar');

주문 건수가 가장 많은 제품군은 Paper임을 알 수 있습니다

In [ ]:
# 주문 수량이 가장 많은 제품군
powermart_df.groupby('Product Sub-Category')['Order Quantity'].sum().sort_values().plot(kind='bar');

주문 수량이 가장 많은 제품군은 Paper임을 알 수 있습니다

In [ ]:
# 매출이 가장 많은 제품군
powermart_df.groupby('Product Sub-Category')['Sales'].sum().sort_values().plot(kind='bar');

반면 매출이 가장 많은 제품군은 Office Machines인 것을 알 수 있습니다  
수량 기준 1등을 차지했던 Paper의 경우 하위권에 위치합니다

In [ ]:
# 수익이 가장 많은 제품군
powermart_df.groupby('Product Sub-Category')['Profit'].sum().sort_values().plot(kind='bar');

- 수익이 가장 많이남는 제품군은 Telehones and Communication, Office Machines, Binders and Binder Accessories으로 볼 수 있습니다    
- Paper는 수익이 나기는 하지만 그 규모가 작은 것을 알 수 있습니다
- 적자가 가장 큰 제품군은 Tables와 Bookcases입니다

### 수익이 가장 많이 남는 상품

수익이 가장 많이남는 제품군인 Telehones and Communication, Office Machines, Binders and Binder Accessories에서 어떤 상품들이 효자인지 찾아봅시다

In [ ]:
# 수익이 많이 남는 TOP3 제품군 만으로 새로운 df를 만듭니다
top3_subcat_df = powermart_df[\
powermart_df['Product Sub-Category'].isin(['Telehones and Communication', 'Office Machines', 'Binders and Binder Accessories']) ]

In [ ]:
top3_subcat_df[['Sales', 'Profit', 'Shipping Cost']].sum()

- TOP3 제품군이 매출에서 차지하는 비중은 3,191,654 / 14,915,600 = 약 21% 가량입니다
- TOP3 제품군이 수익에서 차지하는 비중은 615,126 / 1,521,767 = 약 40% 가량입니다
- TOP3 제품군은 수익성이 좋은 것을 알 수 있습니다.

In [ ]:
# 상품을 기준으로 groupby 해봅니다
product_profit = top3_subcat_df.groupby('Product Name')['Profit'].sum().sort_values(ascending=False)
product_profit

In [ ]:
# 가장 수익이 많이 남는 TOP10 상품
print(product_profit.iloc[:10].sum())
product_profit.iloc[:10]

- TOP10 상품의 수익이 TOP3 제품군에서 차지하는 비중은 371,808 / 615,126 = 무려 60% 가량입니다
- 이들 상품에 대해 강력한 판촉행사를 벌이는 것은 회사의 수익이 좋은 영향일 미칠 것입니다
- Binding System과 Inkjet Printer가 주를 이루고 있습니다

In [ ]:
# 가장 손실이 크게 나는 TOP10 상품
print(product_profit.iloc[-10:].sum())
product_profit.iloc[-10:]

- 수익성이 좋은 제품군 내에서 골랐으므로, 손실이 크게나는 제품들도 손실이 그리 크지는 않습니다
- 그러나 이들 제품에 대한 판매를 줄이거나 중단하는 것은 회사의 손익 개선에 긍정적인 영향을 끼칠 것입니다
- Dot Matrix Printer류가 손실을 이끌고 있습니다

### 어떤 상품이 제일 많이 팔릴까?

In [ ]:
# 판매한 수량 기준
powermart_df.groupby('Product Name')['Order Quantity'].sum().sort_values(ascending=False).iloc[:20]\
.plot(figsize=(12,6), kind='barh');

In [ ]:
# 매출 Sales 기준
powermart_df.groupby('Product Name')['Sales'].sum().sort_values(ascending=False).iloc[:20]\
.plot(figsize=(12,6), kind='barh');

In [ ]:
# 수익 Profit 기준 상위
powermart_df.groupby('Product Name')['Profit'].sum().sort_values(ascending=False).iloc[:20]\
.plot(figsize=(12,6), kind='barh');

In [ ]:
# 수익 Profit 기준 하위
powermart_df.groupby('Product Name')['Profit'].sum().sort_values(ascending=True).iloc[:20]\
.plot(figsize=(12,6), kind='barh');

### 시간에 따른 판매 트렌드는 어떻게 바뀔까?

Product Sub-Category별로 Profit을 시간에 따라 그려보겠습니다

In [ ]:
# groupby를 멀티 레벨로 수행합니다
powermart_df.groupby(['Product Sub-Category', 'Order Date'])['Profit'].sum().reset_index()

In [ ]:
# .pivot_table을 통해 시계열 df 형태로 변경합니다
timeseries_df = pd.pivot_table(powermart_df.groupby(['Product Sub-Category', 'Order Date'])['Profit'].sum().reset_index(),
                               'Profit', 'Order Date', 'Product Sub-Category')
timeseries_df

In [ ]:
# nan을 0으로 채우고, 시간에 따른 누적합 cumsum을 해봅니다
timeseries_df.fillna(0).cumsum(axis=0)

In [ ]:
# 이것을 그래프로 그리면, 상품군 별로 시간에 따라 누적 수익이 어떻게 변하는지 그릴 수 있습니다
timeseries_df.fillna(0).cumsum(axis=0).plot(figsize=(13,8));

In [ ]:
# 그래프를 정리합니다
timeseries_df.fillna(0).cumsum(axis=0).plot(figsize=(13,8), cmap=plt.get_cmap('tab20'));
plt.legend(bbox_to_anchor=(1, 1));
plt.title('Cumulative Profit Time Series by Product Sub-Category');

- Tables는 꾸준히 수익성이 안좋은 것을 알 수 있습니다
- Telephones and Communication은 꾸준히 수익성이 좋은 것을 알 수 있습니다
- Binders and Binder Accessories는 계단식으로 특정시기에 판매가 집중되는 것을 알 수 있습니다.
- Copiers and Fax는 시기에 따라 수익성이 들쭉날쭉 한 것을 알 수 있습니다

시계열 수익성을 최근 시기만 집중해서 분석해봅시다

In [ ]:
# 최근 1년치만 그려봅니다 (2015년 이후)
timeseries_df.loc['2015-01-01':].fillna(0).cumsum(axis=0).plot(figsize=(13,8), cmap=plt.get_cmap('tab20'));
plt.legend(bbox_to_anchor=(1, 1));
plt.title('Cumulative Profit Time Series by Product Sub-Category');

- Telephones and Communication은 여전히 꾸준하게 수익성이 좋은 것을 알 수 있습니다. 아마 B2C가 주력이기 때문일 것입니다
- Copiers and Fax는 최근들어 수익성이 그렇게 좋지 않아졌습니다
- Office Machines과 Binders and Binder Accessories는 계단식으로 특정시기에 판매가 집중되는 것을 알 수 있습니다.

위의 그래프를 그리는 과정에서, y축, 분류, 날짜를 함수화 해봅시다  
- 그룹화의 대상을 변경할 수 있도록 합니다
- y축의 값을 선택할 수 있도록 합니다
- 그래프를 그리는 시기를 지정할 수 있도록 합니다

In [ ]:
def draw_timeseries(groupby='Product Sub-Category', value='Profit', from_date='2011-01-01', to_date='2016-12-31'):

    timeseries_df = pd.pivot_table(powermart_df.groupby([groupby, 'Order Date'])[value].sum().reset_index(),
                                   value, 'Order Date', groupby)
    timeseries_df.loc[from_date:to_date].fillna(0).cumsum(axis=0).plot(figsize=(13,8), cmap=plt.get_cmap('tab20'));
    
    plt.legend(bbox_to_anchor=(1, 1));
    plt.title(f'Cumulative {value} Time Series by {groupby}');

In [ ]:
# 만든 함수 테스트
draw_timeseries()

좋은 함수를 만들었으니, 여러 각도에서 한번 분석해봅시다

  Order Priority 별로 매출과 수익 분석하기

In [ ]:
draw_timeseries('Order Priority', 'Sales')

In [ ]:
draw_timeseries('Order Priority', 'Profit')

- Order Priority간에 매출은 차이가 잘 보이지 않습니다
- Order Priority간에 수익성 차이가 어느정도 있는 것을 확인할 수 있습니다
- 최근들어 'High' Order Priority가 수익성이 좋아졌습니다

  Ship Mode 별로 매출과 수익 분석하기

In [ ]:
draw_timeseries('Ship Mode', 'Sales')

In [ ]:
draw_timeseries('Ship Mode', 'Profit')

- Regular Air와 Express Air는 비슷한 흐름을 보여줍니다 (기울기의 차이는 단순히 volume의 차이입니다)
- Delivery Truck은 매출 대비 수익성이 아주 좋지 않습니다. 아마 Tables와 Funiture등을 이송하기 때문일 것입니다

  지역군 별로 매출과 수익 분석하기

In [ ]:
draw_timeseries('Region', 'Sales')

In [ ]:
draw_timeseries('Region', 'Profit')

- 남부지역의 경우 수익성이 아주 좋은것을 확인할 수 있습니다
- 동부와 서부지역은 수익성이 엎치락 뒤치락 하고있습니다

In [ ]:
draw_timeseries('Region', 'Sales', '2015-01-01')

In [ ]:
draw_timeseries('Region', 'Profit', '2015-01-01')

- 최근 년도에 서부 지역의 수익성이 악화되었습니다

포장 종류 별로 매출과 수익 분석하기

In [ ]:
draw_timeseries('Product Container', 'Sales')

In [ ]:
draw_timeseries('Product Container', 'Profit')

- Jumbo Box로 포장한 상품들의 수익성이 아주 좋지 않습니다
- Large Box로 포장한 상품의 경우 최근들어 수익성이 악화되고 있습니다
- Small Box로 포장한 상품들이 수익성이 좋습니다. 아마 Telehones and Communication와 같은 고수익 상품들이 해당할 것입니다

### 주(State)별로 수익성이나 판매량 차이가 크게 날까?

In [ ]:
# 주별 영업이익률
state_opm = \
(powermart_df.groupby('State')['Profit'].sum() / powermart_df.groupby('State')['Sales'].sum()).sort_values()

In [ ]:
state_opm.describe()

In [ ]:
state_opm

- 평균적인 주별 영업이익률은 10% 가량입니다.
- 수익성이 가장 좋은 주는 [South Dakota, Louisiana, North Dakota] 이며 20%를 상회하는 영업이익률을 자랑합니다
- 반면 수익성이 가장 나쁜 주는 Montana 주이며, -12%로 적자를 보고 있습니다
- 각각의 경우 어떠한 제품군이 많이 팔리는지 알아보겠습니다

In [ ]:
powermart_df[powermart_df.State.isin(['South Dakota', 'Louisiana', 'North Dakota'])]\
.groupby('Product Sub-Category').Profit.sum().sort_values(ascending=False)

In [ ]:
powermart_df[powermart_df.State.isin(['South Dakota', 'Louisiana', 'North Dakota'])]\
.groupby('Product Sub-Category').Profit.sum().sort_values(ascending=False).plot(figsize=(10,5),kind='barh')
plt.title('Category Profit of Best Profitable States');

In [ ]:
powermart_df[powermart_df.State.isin(['Montana'])]\
.groupby('Product Sub-Category').Profit.sum().sort_values(ascending=False).plot(figsize=(10,5),kind='barh')
plt.title('Category Profit of Worst Profitable States');

- 아주 신기한 결과가 나왔습니다. Best 주와 Worst 주 모두 가장 큰 영향을 주는 제품군은 Office Machines 입니다
- Worst 주에서는 Office Machines를 적자를 보며 판매하고 있을 것입니다. 아마 타사와 경쟁중일 것으로 예측해봅니다
- Best 주에서는 Telephone and Communication 보다도 Office Machines의 수익이 높습니다
- 각자의 경우와 시장상황에서 최적의 판촉 전략을 세워볼 수 있을 것 같습니다